In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from IPython.display import display
import json

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36r'}
baseurl = "https://floralimited.com";

In [ ]:
r=requests.get(baseurl + "/brands/acer/desktop")
soup = BeautifulSoup(r.content, 'lxml')

In [ ]:
brands = soup.find('select', {'id': 'BrandID'}).findAll('option')

In [ ]:
empty_file = True
product_list = []
for brand in brands:
    brand_name = brand.text #attr
    included_brands = ['Zebra']
    brand_id = brand.attrs['value']
    r=requests.post(baseurl + "/product/_catagoriesbybrandid", {'id': brand_id})
    soup = BeautifulSoup(r.content, 'lxml')
    categories = json.loads(soup.text)
    for category in categories:
        category_name = category['Text'] #attr
        category_id = category['Value']
        if category_name == 'All in One' and len(categories) > 1:
            continue
        r=requests.post(baseurl + "/product/_gridclient", {'Cataid': category_id, 'BrandId': brand_id, 'StokStatus':'All'})
        soup = BeautifulSoup(r.content, 'lxml')
        products = soup.findAll('div', {'class':'norightpadding'});
        product_list = []
        counter = 0;
        print(brand_name, category_name)
        for product in products:
            label_div = product.find('div', {'class': 'ribbon'})
            product_label = label_div.text if label_div else None #attr
            product_div = product.find('div', {'class': 'product-block'})
            image_div = product_div.find('div', {'class': 'image'})
            product_link = image_div.find('a', href=True).attrs['href'] #attr
            product_image = image_div.find('img').attrs['data-src'] #attr
            info_div = product_div.find('div', {'class': 'product-meta'})
            product_name = info_div.find('div', {'class': 'name'}).text #attr
            product_spec = info_div.find('div', {'class': 'spec'}).text #attr
            product_big_price_old_div = info_div.find('div', {'class': 'big-price'}).find('span', {'class': 'price-old'})
            product_big_price_new_div = info_div.find('div', {'class': 'big-price'}).find('span', {'class': 'price-new'})
            product_big_price = product_big_price_old_div.text if product_big_price_old_div else product_big_price_new_div.text #attr
            product_small_price = info_div.find('div', {'class': 'small-price'}).text #attr
            product_rating_div = info_div.find('div', {'class': 'rating'})
            product_rating = len(product_rating_div.findAll('i', {'class': 'fa-star'})) + (len(product_rating_div.findAll('i', {'class': 'fa-star-half-o'})) * 0.5) #attr
            product = {
                'category_name': category_name,
                'brand_name': brand_name,
                'label': product_label,
                'link': product_link,
                'image': product_image,
                'name': product_name,
                'spec': product_spec,
                'big_price': product_big_price,
                'small_price': product_small_price,
                'rating': product_rating
            }
            r=requests.get(baseurl + product_link)
            soup = BeautifulSoup(r.content, 'lxml')
            info_div = soup.find('div', {'class': 'tab-content'})
            if info_div != None:
                info_div = info_div.find('div', {'id': 'details-info'})
                product_description_title = info_div.find('p').text #attr
                product_description = info_div.find('table').text #attr
                product['description_title'] = product_description_title
                product['description'] = product_description
                counter += 1
                print(counter)
            product_list.append(product)
        df = pd.DataFrame(product_list)
        df.to_csv('floralimited_2021101523_1.csv', mode='a', header=empty_file)
        empty_file = False
        print(category_name, '=>', brand_name)